## Задание

Оформите ваш код из последнего задания первого занятия так, чтобы он представлял собой работу с моделью. Добавьте верификацию модели по тестовой выборке, как в последнем примере.

**Решение:** 

Реализуем полный цикл подготовки данных и обучения модели для полиномиальной регрессии 2-й степени

In [1]:
import torch
import numpy as np

from torch import optim, nn
from torch.utils.data.dataset import random_split
from torch.utils.data import Dataset, TensorDataset, DataLoader

In [2]:
def generate_regression_dataset(device):
    sz = 100
    x = np.random.rand(sz, 1)
    y = 1 + 2 * x + 0.1 * np.random.randn(sz, 1)
    idx = np.arange(sz)
    np.random.shuffle(idx)
    sz80 = (int)(sz*0.8)
    train_idx = idx[: sz80]
    val_idx = idx[sz80:]
    x_train, y_train = x[train_idx], y[train_idx]
    x_val, y_val = x[val_idx], y[val_idx]
    return x_train, y_train


class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        super().__init__()
        self.x = x_tensor
        self.y = y_tensor

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.x)


class PolinomialRegression2ndDegree(nn.Module):
    def __init__(self):
        super().__init__()
        self.a = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        self.b = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        self.c = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))

    def forward(self, x):
        return self.a + self.b * x + self.c * x**2


def make_train_step(model, loss_fn, optimizer):
    # Формируем функцию, которая выполнит один шаг обучения
    def train_step(x, y):
        # Переводим модель в режим обучения
        model.train()
        # Вычислаем прогноз
        yhat = model(x)
        # Считаем лосс
        loss = loss_fn(yhat, y)
        # Вычисляем градиенты
        loss.backward()
        # Обновляем параметры и обнуляем градиенты
        optimizer.step()
        optimizer.zero_grad()
        # Возвращаем лосс
        return loss.item()

    # Возвращаем функцию для вызова внутри цикла обучения
    return train_step

## 0. Configs

In [3]:
# Configs
LR = 0.1
N_EPOCHS = 1000
TRAIN_SET_SIZE_PERC = 0.80
VAL_SET_SIZE_PERC = 0.20
TRAIN_BATCH_SIZE = 16
VAL_BATCH_SIZE = 20
SHUFFLE = True

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

np.random.seed(42)
torch.manual_seed(42)

## 1. Подготовка исходных данных

In [4]:
# Initialize data
x_train, y_train = generate_regression_dataset(DEVICE)

x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train).float()
dataset = TensorDataset(x_train_tensor, y_train_tensor)

print(dataset[0])

(tensor([0.7713]), tensor([2.4745]))


In [5]:
# Prepare Train and Val sets
train_dataset, val_dataset = random_split(dataset, [TRAIN_SET_SIZE_PERC, VAL_SET_SIZE_PERC])

train_loader = DataLoader(dataset=train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=SHUFFLE)
val_loader = DataLoader(dataset=val_dataset, batch_size=VAL_BATCH_SIZE)

## 2. Обучение модели

In [6]:
# Initialize model
model = nn.Sequential(PolinomialRegression2ndDegree()).to(DEVICE)
loss_func = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=LR)

# Fit model
train_step = make_train_step(model, loss_func, optimizer)
for epoch in range(N_EPOCHS):
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)
        loss = train_step(x_batch, y_batch)
    with torch.no_grad():
        for x_val, y_val in val_loader:
            x_val = x_val.to(DEVICE)
            y_val = y_val.to(DEVICE)
            model.eval()
            yhat = model(x_val)
            val_loss = loss_func(yhat, y_val)

print(model.state_dict())
print()
print("Best train loss:", loss)
print("Best val loss:", val_loss)

OrderedDict([('0.a', tensor([1.0498], device='cuda:0')), ('0.b', tensor([1.7781], device='cuda:0')), ('0.c', tensor([0.1830], device='cuda:0'))])

Best train loss: 0.007851201109588146
Best val loss: tensor(0.0070, device='cuda:0')
